In [1]:
################## EDIT THIS TO START ################## 
microbot_input_video_name = "disso1.avi"
output_Video_name = "output2.avi"
diameter_of_cells = 13 # Must be an odd number
proccess_all_frame = False # If False we will proccess the first 200 frames, If True process all frames
########################################################

In [2]:
from __future__ import division, unicode_literals, print_function  # for compatibility with Python 2 and 3
from PIL import Image
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import socket


import cv2
import glob
# change the following to %matplotlib notebook for interactive plotting
%matplotlib inline

# Optionally, tweak styles.
mpl.rc('figure',  figsize=(10, 10))
mpl.rc('image', cmap='gray')

In [3]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series  # for convenience

import pims
import trackpy as tp


In [4]:
##reads in sequence of images

# frames = pims.PyAVVideoReader('./videos/'+microbot_input_video_name)


frames = pims.Video('./videos/'+microbot_input_video_name) 
type(frames[0])

deprecated pixel format used, make sure you did set range correctly


pims.frame.Frame

In [5]:
##converts frames to grey scale
@pims.pipeline
def as_grey(frame):
    red = frame[:, :, 0]
    green = frame[:, :, 1]
    blue = frame[:, :, 2]
    return 0.2125 * red + 0.7154 * green + 0.0721 * blue

In [6]:
frames = as_grey(frames)

In [7]:
##batch does tp.locate on each frame
if proccess_all_frame:
    f = tp.batch(frames, diameter_of_cells, minmass=450, invert=True, processes=1)
else:
     f = tp.batch(frames[:201], diameter_of_cells, minmass=450, invert=True, processes=1)

Frame 200: 10 features


In [10]:
##pass in f
## second parameter is maximum displacement
## third is the most frames a particle can disappear before it is forgotten from memory
t = tp.link_df(f, 10, memory=10)

Frame 200: 10 trajectories present.


In [14]:
##goes through and saves each annotated frame as a png
if proccess_all_frame:
    for i in range(len(frames)):
        print(type(f.loc[f["frame"] == i]))
        print(type(frames[i]))
#         tp.annotate(f.loc[f["frame"] == i], frames[i]).figure.savefig("tempframes/f" + str(i))
else:
    for i in range(201):
        print(type(f.loc[f["frame"] == i]))
        print(type(frames[i]))
#         tp.annotate(f.loc[f["frame"] == i], frames[i]).figure.savefig("tempframes/f" + str(i))

<class 'pandas.core.frame.DataFrame'>
<class 'pims.frame.Frame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pims.frame.Frame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pims.frame.Frame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pims.frame.Frame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pims.frame.Frame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pims.frame.Frame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pims.frame.Frame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pims.frame.Frame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pims.frame.Frame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pims.frame.Frame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pims.frame.Frame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pims.frame.Frame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pims.frame.Frame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pims.frame.Frame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pims.frame.Frame'>
<class 'pandas.core.frame

/Users/casey/opt/anaconda3/lib/python3.8/site-packages/pims/pyav_reader.py:232: MethodDeprecationWarning: VideoStream.seek is deprecated.
  self._stream.seek(timestamp + self._first_pts)
/Users/casey/opt/anaconda3/lib/python3.8/site-packages/pims/pyav_reader.py:232: MethodDeprecationWarning: VideoStream.seek is deprecated.
  self._stream.seek(timestamp + self._first_pts)


<class 'pims.frame.Frame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pims.frame.Frame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pims.frame.Frame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pims.frame.Frame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pims.frame.Frame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pims.frame.Frame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pims.frame.Frame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pims.frame.Frame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pims.frame.Frame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pims.frame.Frame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pims.frame.Frame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pims.frame.Frame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pims.frame.Frame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pims.frame.Frame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pims.frame.Frame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pims.frame.Frame'

In [10]:
##gets the annotated frames and turns them into a video
img_array = []
if proccess_all_frame:
    for i in range(1, len(frames)):
        #glob.glob('tempframes/*.png'):
        filename = "tempframes/f" + str(i) + ".png"
        frame = Image.open(filename)
        img = cv2.imread(filename)
        height,width, layers = img.shape
        size = (width,height)
        img_array.append(img)
else:
    for i in range(1, len(frames[:201])):
        #glob.glob('tempframes/*.png'):
        filename = "tempframes/f" + str(i) + ".png"
        frame = Image.open(filename)
        img = cv2.imread(filename)
        height,width, layers = img.shape
        size = (width,height)
        img_array.append(img)
out = cv2.VideoWriter(output_Video_name,cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()

In [11]:
dir = 'tempframes'
filelist = glob.glob(os.path.join(dir, "*"))
for f in filelist:
    os.remove(f)